# Ideas / plans for model serialisation

In [ ]:
from gammapy.spectrum.models import SpectralModel, PowerLaw
from gammapy.cube.models import SkyModel
from gammapy.image.models import SkySpatialModel

## 1. Pull Request

Allow dictionaries for parameter values, to specify `min`, `max` and `frozen` etc. for each parameter on init:

In [ ]:
pwl = PowerLaw(
    amplitude={"value": "1e-12 TeV-1 cm-2 s-2"},
    index={"value": 2.3, "min": 2.0},
    reference={"value": "1 TeV", "frozen": True}
)

This can be achieved by handling the case `if isinstance(factor, dict):` in https://github.com/gammapy/gammapy/blob/master/gammapy/utils/fitting/parameter.py#L56.

## 2. Pull Request
Add `.create()` factory methods to the spectral, spatial and cube base classes. The actual implementation can be the same as https://github.com/gammapy/gammapy/blob/master/gammapy/spectrum/models.py#L221. But instead of calling `Parameters.from_dict()`, pass the dictionaries to `__init__`, as outlined above. Covariance can be ignored for now. The translation from model types to classes is done using `globals()`, which is an acceptable solution for now.

This is how the `.create()` methods are used:

In [ ]:
pwl = SpectralModel.create(type="PowerLaw", amplitude={}, ...)
gauss = SkySpatialModel.create(type="SkyGaussian", lon_0={}, ...)
source = SkyModel.create(
    spatial_type="SkyGaussian",
    spectral_type="PowerLaw",
    name="source-abc",
    lon_0={},
    lat_0={},
    index={},
)

The `SkyModel.create()` calls internally `SkySpatialModel.create()` and `SpectralModel.create()`. To separate the spatial from the spectral parameters one could use the following: 

In [ ]:
spectral_pars = globals()["PowerLaw"].__slots__
kwargs_spectral = {name: kwargs[name] for name in kwargs if name in spectral_pars}

If this seems to complicated, the following pattern could be used as well:

In [ ]:
source = SkyModel.create(
    spatial_type="SkyGaussian",
    spectral_type="PowerLaw",
    name="source-abc",
    spatial_pars={},
    spectral_pars={},
)

## 3. Pull Request

Add a `Parameter.to_yaml_dict()` method:

In [ ]:
pwl.amplitude.to_yaml_dict()

Which should return the following:

In [ ]:
{
    "value": "1.23456e-12 TeV-1 cm-2 s-1",
    "frozen": True,
    "min": "1e-13 TeV-1 cm-2 s-1",
    "max": "1e-10 TeV-1 cm-2 s-1"
}

Entries in this dict are only added if they differ from the defaults:

```
frozen=False
min=np.nan
max=np.nan
```

The separate handling of `scale` and `value` is not needed and should be hidden from the user.

Add a `Parameters.to_yaml_dict()` method:

In [ ]:
pwl.parameters.to_yaml_dict()

Which should return:

In [ ]:
{
    "amplitude": {"value": "1.23456e-12 TeV-1 cm-2 s-1", "frozen": True},
    "index": {"value": 2.3, "frozen": True},
    "reference": {"value": "1 TeV", "frozen": True}
}

## 4.Pull Request

Add a `.to_dict()` method to the Model base class:

In [ ]:
pwl.to_dict()

Which should return:

In [ ]:
{
"type": "PowerLaw",
"name": "source-xyz",
"parameters":
    {
        "amplitude": {"value": "1.23456e-12 TeV-1 cm-2 s-1", "frozen": True},
        "index": {"value": 2.3, "frozen": True},
        "reference": {"value": "1 TeV", "frozen": True}
    }
}

The `SkyModel`, `SkyDiffuseCube` and `SkyDiffuseMap` require as special `.to_dict()` method, to handle filenames and spectral / spatial models.

The `SkyModel.to_dict()` calls `.spectra_model.to_dict()` and `.spatial_model.to_dict()` and should return:

In [ ]:
{
    "spectral-model": {},
    "spatial-model": {}
}

The `SkyModels` class should implement a `.to_list()` method, which calls `SkyModel.to_dict()` in a loop and appends the results:

In [ ]:
models = SkyModels()
models.to_list()

In addition it should implement `.read()` and `.write()`. For now we support only the yaml format. The support for xml can be removed.

In [ ]:
models = SkyModels.read("model.yaml")
model.write("models.yaml")

Finally this should simplify the actual YAML serialization, so that it becomes even more readable / editable by humans. Here an example YAML serialization for spectral model:

```
type: PowerLaw
name: source-xyz
parameters:
    amplitude:
        value: 1e-12 TeV-1 cm-2 s-1
        frozen: False
    index:
        value: 2.3
        min: 2.0
        max: 2.5
        frozen: False
    reference:
        value: 1 TeV
        frozen: True
```

And the same for `SkyModels` objects:

```
sources:
  - name : source-abc
    type: SkyModel
    obs-id: global
    spatial-model:
        type: SkyGaussian
        parameters: 
            lon_0:
                value: 0 deg
                frozen: False
            lat_0:
                value: 0 deg
                frozen: False
            sigma:
                value: 0.1 deg
                frozen: False
                min: 0.05 deg
                max: 0.2 deg
    spectral-model:
        type: PowerLaw
        parameters:
            amplitude:
                value: 1e-12 TeV-s cm-2 s-1
                frozen: False
            index:
                value: 2.3
                min: 2.0
                max: 2.5
                frozen: False
            reference:
                value: 1 TeV
                frozen: True
  - name : source-xyz
    type: SkyDiffuseCube
    filename: diffuse-model.fits
    obs-id: global
    parameters:
        norm:
            value: 1
            frozen: False
         tilt:
            value: 0
            frozen: True
         reference:
             value: 1 TeV
             frozen: True
```